In [1]:
import csv
fi=open('Data/redditSubmissions.csv')
r=csv.reader(fi)
data=list(r)

In [2]:
data.pop(0)

['#image_id',
 'unixtime',
 'rawtime',
 'title',
 'total_votes',
 'reddit_id',
 'number_of_upvotes',
 'subreddit',
 'number_of_downvotes',
 'localtime',
 'score',
 'number_of_comments',
 'username']

In [3]:
data.pop(53033)

['20089', '']

In [48]:
imageIdDict={}

In [55]:
!rm reddit_upvotes_train.dat, reddit_upvotes_test.dat
train=open("reddit_upvotes_train.dat","wb")
test=open("reddit_upvotes_test.dat","wb")

rm: cannot remove ‘reddit_upvotes_train.dat,’: No such file or directory
rm: cannot remove ‘reddit_upvotes_test.dat’: No such file or directory


In [56]:
import datetime
import random
for i in xrange(len(data)):
    d=data[i]
    t=d[2]
    tFormatted=datetime.datetime(int(t[0:4]),int(t[5:7]),int(t[8:10]),
                                     int(t[11:13]),int(t[14:16]),int(t[17:19]))
    s="%s::%s::%s::%s\n"%(d[0],tFormatted.hour,d[6],i)
    if d[0] in imageIdDict:
        imageIdDict[d[0]].append("%s-%s; "%(tFormatted.hour,d[6]))
    else:
        imageIdDict[d[0]]=["%s-%s; "%(tFormatted.hour,d[6])]
    c=random.random()
    if(c<0.8):
        train.write(s)
    else:
        test.write(s)

In [57]:
train.close()
test.close()

In [54]:
imageIdDict['411']

['5-48358; ',
 '11-8; ',
 '14-332; ',
 '11-3492; ',
 '19-75; ',
 '20-16; ',
 '18-29; ',
 '1-44; ',
 '5-9; ',
 '18-28; ',
 '3-18; ',
 '18-12; ',
 '5-22; ',
 '23-91; ',
 '21-16; ',
 '1-84; ',
 '19-9; ']

In [102]:
import recsys.algorithm
recsys.algorithm.VERBOSE = True
from recsys.algorithm.factorize import SVD
svd=SVD()

In [103]:
svd.load_data(filename='reddit_upvotes_train.dat', sep='::', format={'col':0, 'row':1, 'value':2, 'ids': int})

Loading reddit_upvotes_train.dat
.


In [83]:
#svd.compute(k=100, min_values=10, pre_normalize=None, mean_center=True, post_normalize=True)

Creating matrix (105802 tuples)
Matrix density is: 20.7004%
Updating matrix: squish to at least 10 values
Computing svd k=100, min_values=10, pre_normalize=None, mean_center=True, post_normalize=True
[WARNING] mean_center is True. svd.similar(...) might return nan's. If so, then do svd.compute(..., mean_center=False)


In [126]:
imageIdDict[data[20][0]]

['12-32657; ',
 '14-29; ',
 '20-32; ',
 '10-6; ',
 '16-57; ',
 '8-17; ',
 '12-67; ',
 '19-5; ',
 '16-6; ',
 '21-9; ',
 '15-10; ',
 '18-210; ',
 '12-4612; ',
 '13-15; ',
 '20-8; ',
 '3-14; ',
 '18-8; ',
 '6-6; ',
 '12-7; ',
 '23-1; ',
 '12-97; ',
 '4-6; ',
 '4-39; ',
 '21-9; ',
 '23-1; ',
 '20-219; ',
 '2-10; ',
 '12-32657; ',
 '14-29; ',
 '20-32; ',
 '10-6; ',
 '16-57; ',
 '8-17; ',
 '12-67; ',
 '19-5; ',
 '16-6; ',
 '21-9; ',
 '15-10; ',
 '18-210; ',
 '12-4612; ',
 '13-15; ',
 '20-8; ',
 '3-14; ',
 '18-8; ',
 '6-6; ',
 '12-7; ',
 '23-1; ',
 '12-97; ',
 '4-6; ',
 '4-39; ',
 '21-9; ',
 '23-1; ',
 '20-219; ',
 '2-10; ']

In [136]:
svd.predict(20,23)

91.99999946112621

In [147]:
from scipy import stats
preds=[]
testValue=[]
import math
import numpy as np
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import r2_score as r2s
i=1
test=open("reddit_upvotes_test.dat","rb")
for l in test.readlines():
    image,hr,upvotes,rowid = l.split('::')
    #print hr,image
    try:
        predUpvotes=svd.predict(int(hr),int(image))
        preds.append(predUpvotes)
        testValue.append(int(upvotes))
        #mse+=(predUpvotes-int(upvotes))**2
    except:
        continue
    #scaledValue=predUpvotes*1.0/int(upvotes)
    #mse+=scaledValue*scaledValue-1
    i+=1
    
#rmse=np.sqrt(mse*1.0/i)
#print rmse,i,mse
slope, intercept, r_value, p_value, std_err = stats.linregress(testValue,preds)
print np.mean(preds),np.std(preds),np.mean(testValue),np.std(testValue)
print r2s(preds,testValue), mse(testValue,preds)**0.5
test.close()

423.355713947 2039.74082702 1032.84987875 3085.68461119
-2.38994024832 3755.52805275


In [72]:
!cat reddit_upvotes_test.dat

0::20::32::2
0::16::57::4
0::12::67::6
0::21::9::23
0::2::10::26
1::18::60::27
1::4::2::28
1::3::92::29
100::19::11::37
100::14::4::42
1000::5::37::45
10001::14::31::50
10001::18::45::51
10001::7::81::52
10002::17::5::60
10005::10::2664::70
10005::8::41::74
10005::6::33::75
10005::16::1276::82
10005::15::180::83
10005::5::436::84
10005::11::63::87
10005::20::29::89
10005::9::3::94
10005::14::655::98
10007::23::19::104
10007::3::1::106
1001::18::62::120
1001::17::176::124
1001::13::25::127
1001::8::94::137
1001::5::37::142
1001::20::6::143
1001::16::14::144
1001::4::7::150
1001::11::52::152
10010::22::31::154
10011::21::24::157
10011::3::4::159
10013::19::42::162
10014::14::2212::165
10014::14::8842::166
10014::7::6::174
10014::21::15::175
10015::6::14::183
10016::12::46::190
10016::19::22::191
10016::11::53::192
10016::14::1::193
10016::18::16::197
10016::23::15::198
10016::1::16::201
10016::13::59::203
10016::20::11::205
10016::19::2::206
10017::19::90::207
10018::15::100::212
10018::

In [39]:
svd.get_matrix()

Creating matrix (132307 tuples)
Matrix density is: 24.6783%


SparseMatrix (24 by 16736)
         0          1          100        1000       10001    ...
12       3.74e+04      ---     5.000000      ---        ---
14      29.000000      ---     4.000000      ---    31.000000
20       2.59e+02      ---        ---        ---     1.19e+02
10       6.000000      ---     2.000000   3.000000      ---
16       6.30e+01      ---    11.000000      ---        ---
8       17.000000      ---        ---        ---        ---
19       5.000000      ---    11.000000      ---        ---
21      18.000000      ---        ---        ---    11.000000
15      10.000000      ---        ---     8.000000      ---
18       2.18e+02   6.00e+01   8.000000      ---    45.000000
13      15.000000      ---        ---        ---        ---
3       14.000000   9.20e+01      ---        ---        ---
6        6.000000      ---        ---        ---        ---
23       2.000000      ---    16.000000      ---        ---
4       45.000000   2.000000      ---        ---        ---

In [40]:
svd.recommend(5) #best posts to submit at hour 5

Creating matrix (132307 tuples)
Matrix density is: 24.6783%
Computing svd k=100, min_values=None, pre_normalize=None, mean_center=False, post_normalize=True
[WARNING] min_values is set to None, meaning that some funky recommendations might appear!


/usr/lib/python2.7/dist-packages/pkg_resources.py:1031: UserWarning: /home/sachin/.python-eggs is writable by group/others and vulnerable to attack when used with get_resource_filename. Consider a more secure location (set with .set_extraction_path or the PYTHON_EGG_CACHE environment variable).
  warnings.warn(msg, UserWarning)


[(411, 48388.999999992564),
 (13274, 33797.000000000349),
 (13920, 32504.000000000167),
 (12608, 29959.999999974691),
 (19616, 29804.999999999876),
 (505, 27559.000000000084),
 (15147, 26327.999999999753),
 (1883, 26109.000000000346),
 (10428, 25312.000000006727),
 (6454, 25129.999999995271)]

In [44]:
svd.recommend(411, is_row=False) #best hours to submit for post 411

[(5, 48388.999999992564),
 (11, 3499.9999998506696),
 (14, 331.99999997509889),
 (1, 127.99999999708646),
 (23, 90.999999984499155),
 (19, 84.000000101825364),
 (18, 68.999999855748698),
 (3, 17.999999990687584),
 (20, 15.999999995080806),
 (21, 15.999999985010403)]